In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings


In [2]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [12]:
# List of CSV files to merge
csv_files = [
    "data/_2010/_labelled/heat_labelled.csv",
    "data/_2011/_labelled/heat_labelled.csv",
    "data/_2012/_labelled/heat_labelled.csv",
    "data/_2013/_labelled/heat_labelled.csv",
    "data/_2014/_labelled/heat_labelled.csv",
    # "data/_2015/_labelled/heat_labelled.csv",
    # "data/_2016/_labelled/heat_labelled.csv",
    # "data/_2017/_labelled/heat_labelled.csv",
    # "data/_2018/_labelled/heat_labelled.csv",
    "data/_2019/_labelled/heat_labelled.csv", 
    "data/_2020/_labelled/heat_labelled.csv",
    "data/_2021/_labelled/heat_labelled.csv",
    "data/_2022/_labelled/heat_labelled.csv",
    "data/_2023/_labelled/heat_labelled.csv",
    "data/_2024/_labelled/heat_labelled.csv",
    ]  # Add more files if needed

# Read and merge files
df_list = [pd.read_csv(file) for file in csv_files]  # Read each CSV
new_merged_df = pd.concat(df_list, ignore_index=True)
new_merged_df = new_merged_df.drop(columns=["Unnamed: 0"])

In [13]:
new_merged_df

,latitude,longitude,max_temperature,mean_temperature,max_precipitation,mean_precipitation,mslp,month,day_of_year,year,heatwave_label
0,18.80,72.8,300.55830,298.53610,0.000000e+00,0.000000e+00,101107.070,1,1,2010,0
1,19.05,72.8,301.07275,298.24356,0.000000e+00,0.000000e+00,101114.100,1,1,2010,0
2,18.80,72.8,301.43620,299.36893,3.810000e-07,1.590000e-08,101242.380,1,2,2010,0
3,19.05,72.8,301.72270,299.18018,0.000000e+00,0.000000e+00,101246.875,1,2,2010,0
4,18.80,72.8,300.03796,298.97437,1.910000e-07,7.940000e-09,101397.836,1,3,2010,0
...,...,...,...,...,...,...,...,...,...,...,...
8031,19.05,72.8,300.16570,298.23690,0.000000e+00,0.000000e+00,101466.620,12,364,2024,0
8032,18.80,72.8,300.97717,298.98740,0.000000e+00,0.000000e+00,101442.690,12,365,2024,0
8033,19.05,72.8,301.34230,299.03720,0.000000e+00,0.000000e+00,101448.500,12,365,2024,0
8034,18.80,72.8,300.60540,299.00990,0.000000e+00,0.000000e+00,101487.400,12,366,2024,0


In [14]:
if os.path.exists("data/_11year_heat_labelled.csv"):
    print("Files already exist")
else:
    new_merged_df.to_csv("data/_11year_heat_labelled.csv")
    print("File created")

File created


In [20]:
df = pd.read_csv("data/_11year_heat_labelled.csv")

In [21]:
# df = df.drop(columns=['Unnamed:0'], axis=1)
df

,Unnamed: 0,latitude,longitude,max_temperature,mean_temperature,max_precipitation,mean_precipitation,mslp,month,day_of_year,year,heatwave_label
0,0,18.80,72.8,300.55830,298.53610,0.000000e+00,0.000000e+00,101107.070,1,1,2010,0
1,1,19.05,72.8,301.07275,298.24356,0.000000e+00,0.000000e+00,101114.100,1,1,2010,0
2,2,18.80,72.8,301.43620,299.36893,3.810000e-07,1.590000e-08,101242.380,1,2,2010,0
3,3,19.05,72.8,301.72270,299.18018,0.000000e+00,0.000000e+00,101246.875,1,2,2010,0
4,4,18.80,72.8,300.03796,298.97437,1.910000e-07,7.940000e-09,101397.836,1,3,2010,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8031,8031,19.05,72.8,300.16570,298.23690,0.000000e+00,0.000000e+00,101466.620,12,364,2024,0
8032,8032,18.80,72.8,300.97717,298.98740,0.000000e+00,0.000000e+00,101442.690,12,365,2024,0
8033,8033,19.05,72.8,301.34230,299.03720,0.000000e+00,0.000000e+00,101448.500,12,365,2024,0
8034,8034,18.80,72.8,300.60540,299.00990,0.000000e+00,0.000000e+00,101487.400,12,366,2024,0


In [22]:
df = df.drop(columns=['Unnamed: 0'], axis=1)


In [23]:
X = df.drop(columns=['heatwave_label'], axis=1)
X

,latitude,longitude,max_temperature,mean_temperature,max_precipitation,mean_precipitation,mslp,month,day_of_year,year
0,18.80,72.8,300.55830,298.53610,0.000000e+00,0.000000e+00,101107.070,1,1,2010
1,19.05,72.8,301.07275,298.24356,0.000000e+00,0.000000e+00,101114.100,1,1,2010
2,18.80,72.8,301.43620,299.36893,3.810000e-07,1.590000e-08,101242.380,1,2,2010
3,19.05,72.8,301.72270,299.18018,0.000000e+00,0.000000e+00,101246.875,1,2,2010
4,18.80,72.8,300.03796,298.97437,1.910000e-07,7.940000e-09,101397.836,1,3,2010
...,...,...,...,...,...,...,...,...,...,...
8031,19.05,72.8,300.16570,298.23690,0.000000e+00,0.000000e+00,101466.620,12,364,2024
8032,18.80,72.8,300.97717,298.98740,0.000000e+00,0.000000e+00,101442.690,12,365,2024
8033,19.05,72.8,301.34230,299.03720,0.000000e+00,0.000000e+00,101448.500,12,365,2024
8034,18.80,72.8,300.60540,299.00990,0.000000e+00,0.000000e+00,101487.400,12,366,2024


In [24]:
y = df['heatwave_label']
y

0       0
1       0
2       0
3       0
4       0
       ..
8031    0
8032    0
8033    0
8034    0
8035    0
Name: heatwave_label, Length: 8036, dtype: int64

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((6428, 10), (1608, 10))

In [26]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [27]:
models = {
    "Ridge": RidgeClassifier(),
    "K-Neighbors Classifier": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest Classifier": RandomForestClassifier(),
    "XGBClassifier": XGBClassifier(), 
    "CatBoosting Classifier": CatBoostClassifier(verbose=False),
    "AdaBoost Classifier": AdaBoostClassifier(),
    "SVM": SVC()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('\n')


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.3012
- Mean Absolute Error: 0.0907
- R2 Score: 0.0049
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.2940
- Mean Absolute Error: 0.0864
- R2 Score: 0.0018


K-Neighbors Classifier
Model performance for Training set
- Root Mean Squared Error: 0.2655
- Mean Absolute Error: 0.0705
- R2 Score: 0.2268
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.3280
- Mean Absolute Error: 0.1076
- R2 Score: -0.2424


Decision Tree
Model performance for Training set
- Root Mean Squared Error: 0.0000
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.0432
- Mean Absolute Error: 0.0019
- R2 Score: 0.9785


Random Forest Classifier
Model performance for Training set
- Root Mean Squared Error: 0.0000
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000
------

In [28]:

pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)


,Model Name,R2_Score
2,Decision Tree,0.978456
4,XGBClassifier,0.971275
5,CatBoosting Classifier,0.964094
3,Random Forest Classifier,0.842012
6,AdaBoost Classifier,0.791743
0,Ridge,0.001804
7,SVM,-0.105915
1,K-Neighbors Classifier,-0.242359
